In [ ]:
### What do I need? 

# 1. Function mapping from Gyro rates (Body frame angular rates) to Quaternion rates: 

# 2. Function mapping from Gyro rates (Body frame angular rates) to Euler angle rates: 

# 3. Function that converts a vector to skew symmetric matrix: DONE!

# 4. Function to perform numerical integration for a vector: Runge-Kutta ?? Refer to what Dr Psiaki used here: 
#    Given: An initial vector, the time derivative, start time and final time. Dr Psiaki uses Runge-Kutta 4th order
#    Use his implementation as a reference. Lambda function used to return x_dot, dx_dot/dx , dx_dot/du

# 5. Function to compute rotation matrix from quaternion: DONE!

# 6. Functions to compute : s_l and y_l : TO DO!

# 7. Function to propagate IMU quaternion given angular velocity measurements:

# 10. Functions to check the gradients of error propagation partial derivatives: Maybe Later!





# -------------------------------------- #
# Params file: 

# filtDelt  - Filter time step 

# nRK - Number of Runge-Kutta steps per filter step

# 



In [2]:
### Import libraries: 
import numpy as np

In [7]:
### 3. Convert vector to skew symmetric matrix:
def vect_2_skew(vect): 
    
    """
    Input: 
        - vect - 3 x 1 vector: 
    
    Output: 
        - skew - 3 x 3 skew symmetric matrix as a NumPy 2D array:
    """
    
    ## Assert that vect is a 3 x 1 vector: 
    
    assert vect.shape == (3,1) or vect.shape == (1,3) or vect.shape == (3,)
    
    # If it is a row vector then transpose first:
    if vect.shape == (1,3):         
        vect = vect.T
      
    
    # Construct the skew symmetric matrix:    
    skew = np.array([[0  , -vect[2] , vect[1]] , 
                     [-vect[2] , 0 , -vect[0]], 
                     [-vect[1] , vect[0],  0 ] ] )
    
    return skew

### Test: 

vect = np.array([-2, 3,6])

skew = vect_2_skew(vect)

print (skew)

[[ 0 -6  3]
 [-6  0  2]
 [-3 -2  0]]


In [8]:
### 5. Function to get rotation matrix from Quaternion: 
def quat_2_rot(q): 
    
    
    rot = np.zeros((3,3), dtype = np.float64)
    
    # 1st row: 
    rot[0][0] = q[0]**2 - q[1]**2 - q[2]**2 + q[3]**2
    rot[0][1] = 2*(q[0]*q[1] + q[2]*q[3]) 
    rot[0][2] = 2*(q[0]*q[2] - q[1]*q[3])
    
    # 2nd row: 
    rot[1][0] = 2*(q[0]*q[1] - q[2]*q[3]) 
    rot[1][1] = -q[0]**2 + q[1]**2 - q[2]**2 + q[3]**2
    rot[1][2] = 2*(q[1]*q[2] + q[0]*q[3]) 
    
    # 3rd row: 
    rot[2][0] = 2*(q[0]*q[2] + q[1]*q[3])
    rot[2][1] = 2*(q[1]*q[2] - q[0]*q[3])
    rot[2][2] = -q[0]**2 - q[1]**2 + q[2]**2 + q[3]**2
    
    
    return rot
    
    
### Test: 
q = np.array([0,0,0.5,np.cos(np.pi/6)])

R1 = quat_2_rot(q)
    
import transforms3d as t3d

### Test : 
R2 = t3d.quaternions.quat2mat(q)

print (R1)
print (R2)

## Conclusion: Need to use custom function!!!

[[ 0.5        0.8660254  0.       ]
 [-0.8660254  0.5        0.       ]
 [ 0.         0.         1.       ]]
[[-1.         0.         0.       ]
 [ 0.        -0.5        0.8660254]
 [ 0.         0.8660254  0.5      ]]


In [9]:
### 7. Compute time derivative of quaternion given body frame angular velocities: 

def q_dot(t,q,omega): 
    
    """
    Inputs: 
        - t -> Current time value (secs)
        - q -> Quaternion at current time step - (4,1)
        - omega -> Body frame angular velocities at current time step - (3,1)
    
    Outputs: 
    
    """
    
    # Assert that shape of omega is (3,) or at least (1,3)
    
    
    # Reshape omega to a column vector: 
    omega = np.reshape(omega, (3,1))
    
    # Construct Big Omega: 
    bOmega = np.hstack( (np.vstack( (-1*vect_2_skew(omega) , -1*omega.T) ), np.vstack((omega , 1)) ) )
    
    # TO D0 : Assert that q is a (4,1) vector: 
    if q.shape != (4,1): 
        
        try:
            q = np.reshape(q,(4,1))
            
        except ValueError: 
            
            print (" !!! Quaternion Dimension Error !!! ")
            print (" Make sure that the number of elements in the quaternion is 4 ")
            print (" Current Number of elements is: " , len(q))
    
    # Return out q_dot: 
    return 0.5*np.dot(bOmega , q)
                       
                       
                       
                       
# Test: 
omega = np.array([2,3,4])

q = np.array([[0,0,0.5,0.8666]])
t= 0.5
                       
q_dot(t, q,omega)


array([[ 0.1166],
       [ 1.7999],
       [ 1.7332],
       [-0.5667]])

In [10]:
### 7.b Compute time derivative of velocity : v_dot: 
def v_dot_pre(a_b, g_g, q_g2b): 
    
    """
     Inputs: 
         - a_b -> Acceleration in Body frame (m/s^2) - From IMU
         - g_g -> Acceleration due to gravity in global frame (m/s^2)
         - q_g2b -> Quaternion that parameterizes rotation from global to body frame. 
     
     Outputs: 
         - v_dot -> Time derivative of velocity 
     """
    
    # Time derivative of velocity is acceleration is global frame minus the acceleration due to gravity: 
    
    # Extract rotation matrix from quaternion vector: Rot Matrix from Global to Body frame:
    R_g2b = quat_2_rot(q_g2b)
    
    # Assert that the acceleration in body frame is a (3,1) vector: 
    if a_b.shape != (3,1): 
        
        a_b = np.reshape(ab_, (3,1))
     
    # Acceleration in Global frame: 
    a_g = np.dot(R_g2b.T , a_b)   # Transpose is because we want rotation from body to global
    
    return a_g - g_g


def v_dot(t, v,a_g): 
    
    return a_g

In [11]:
### 7.c : Compute time derivative of position: p_dot: 
def p_dot(t,p,v): 
    
    """
    
    Inputs: 
    
    
    Outputs: 
    
    
    """
    
    return v
    
    

In [13]:
### 8. Function that performs Runge-Kutta 4th order integration: Generic version: 
def rk4_core(x0 ,u, t0 , delT, x_dot_func):
    
    """
    Inputs: 
        - x0 -> Value of the vector to be integrated at time t0 secs
        - u  -> Input vector driving the system. 
        - t0 -> Initial time (secs)
        - tf -> Final time (secs)
        - x_dot_func -> Name of the function which computes time derivative
        
    Outputs: 
        - xf -> Value of the integrated variable at the final time tf 
        
    Logic: 
        - The RK function computes 4 slopes evaluated at points between t0 and tf. It then uses the weighted average of these
        slopes to predict the value of x at time tf using the formula: 
        # ---   x(tf) = x(t0) + (1/6)*(k1 + 2*k2 + 2*k3 + k4)*(tf - t0)  --- #
        
    """
    
    
    # k1 -->Compute the time derivative (a.k.a slope) at initial time t0:    
    k1 = x_dot_func(t,x0, u)
    

    # k2 --> Compute the time derivative at mid-point using k1 to get value of x at mid point:    
    k2 = x_dot_func( t + delT/2 , x0 + k1*delT/2 , u)   # Input vector u is kept constant throughout the sample interval
    
    
    # k3 --> Once again compute derivative at mid-point but use k2 to get value of x at mid point : 
    k3 = x_dot_func( t + delT/2 , x0 + k2*delT/2 , u )
    
    # k4 --> Compute derivative at end-point using k3 to get value of x at end-point: 
    k4 = x_dot_func( t + delT, x0 + k3*delT , u)
    
    
    # Finally predict the value of the state at final point: 
    return x0 + (1/6)*(k1 + 2*k2 + 2*k3 + k4)*delT
    
        

### Test: Quaternion Propagation:
q0 = np.array([0,0,0.5,0.86666]).reshape((4,1))

omega = np.array([0.2,0.3,0.15]).reshape((3,1))

t0 = 0.5

# Call the RK function: 
out = rk4_core(q0, omega,t0 , 0.5, q_dot)

print (out)

### Test: Velocity Propagation:

v0 = np.array([0,0,0.5]).reshape((3,1))

a_b = np.array([0.5,0.23, 10.23]).reshape((3,1))

q_g2b = np.array([0,0,0.5,0.86666]).reshape((4,1))

g_g = np.array([0,0,9.81]).reshape((3,1))

a_g = v_dot_pre(a_b, g_g, q_g2b)


vf = rk4_core(v0, a_g, t0, 0.5, v_dot)


### Test : Position Propagation: 

p0 = np.array([0.2, 0.4, 1]).reshape((3,1))

pf = rk4_core(p0, vf, t0 , 0.5 , p_dot)

   
    

[[ 0.01167385]
 [ 0.09898729]
 [ 0.53444645]
 [ 1.08665021]]


In [24]:
### Runge Kutta Integration Wrapper function: 

def rk4_wrapper(x0, u, t0, tf, x_dot_func, nRK):

    delT = (tf - t0)/nRK
    
    currTime = t0
    
    while currTime < tf: 
        
        #print (" Runge-Kutta Iter: Time = " , currTime, " secs")
        
        xf = rk4_core(x0,u,currTime, delT, x_dot_func)
        
        # Pass on the new variables to the old ones: 
        x0 = xf
        currTime = currTime + delT           
    
    # Finally return out the final value of the variable: 
    return xf

### Test this out: 
q0 = np.array([0,0,0.5,0.86666]).reshape((4,1))

omega = np.array([0.2,0.3,0.15]).reshape((3,1))

t0 = 0.5

qf = rk4_wrapper(q0, omega, t0, 1, q_dot, 21)

print (qf)

[[ 0.01167461]
 [ 0.09898863]
 [ 0.53444692]
 [ 1.08665294]]


In [1]:
### Error Propagation: TO DO: ETC: 21st Feb 2018: 

### Goal: Compute State Transition Matrix(Phi) for the Error states: Divide states into : 1) Quaternion 2) Velocity and 3) Position. 

### Compute : 1) Phi_qq = R_bar*R_hat  2) Phi_vq  = - [s_lx]*R_hat.T  3) Phi_pq = -[y_l x]*R_hat

### TO BE TESTED!!!

def get_error_state_transition(v_hat, v_bar, p_hat, p_bar, R_hat, R_bar, delT, g_g): 
    
    # Ensure that: v_hat, v_bar , p_hat, p_bar and g_g are (3,1)
    
    # Step 1: Construct Phi_qq - Shape (3,3)
    
    Phi_qq = np.dot(R_bar, R_hat.T)   # Shape (3,3) = (3,3) x (3,3)
    

    
    # Step 2: Construct Phi_vq - Shape (3,3): 
    
    #s_l = np.dot(R_hat, (v_bar - v_hat - g_g*delT))
    
    Phi_vq = np.dot( -1*vect_2_skew(v_bar - v_hat - g_g*delT) , R_hat.T)
    
    # Step 3: Construct Phi_pq - Shape (3,3): 
    
    # y_l = ??
    
    Phi_pq  = np.dot( -1*vect_2_skew(p_bar - p_hat - v_hat*delT - 0.5*g_g*delT**2) , R_hat.T)
    
    
    # Now assemble the entire error state transition matrix: 
    Phi = np.identity(9)
    
    Phi[0:3,0:3] = Phi_qq
    Phi[3:6,0:3] = Phi_pq 
    Phi[6:9, 0:3]= Phi_vq
    
    Phi[3:6,6:9] = delT*np.identity(3)
    
    
    return Phi


In [ ]:
### Numerical Integration of the Continous-time process noise matrix : Discretization step: First refer to Dr Psiaki's note
#   on this. 

In [ ]:
### Update Step: 

### Get measurement residuals as a function of error state instead of true state: H - (3,9) instead of (3,10)

## Output from update step is: Updated Estimate of the error in the state. Add it to the state vector. 